In [ ]:
import os
import json
import re
from dotenv import load_dotenv
from openai import AzureOpenAI, OpenAIError

load_dotenv()  # .env 파일에 있는 환경변수 로드

aoi_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoi_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoi_gen_model = os.getenv("AZURE_GENERATION_MODEL")
aoi_version = os.getenv("AZURE_GENERATION_MODEL_VERSION")
client = AzureOpenAI(azure_endpoint=aoi_endpoint,api_key=aoi_api_key, api_version=aoi_version)


def get_story_from_openai(prompt: str) -> str:
    # 기존 openai.ChatCompletion.create 대신 AzureOpenAI client 사용
    completion = client.chat.completions.create(
        model=aoi_gen_model,  # custom deployment name
        messages=[
            {"role": "system", "content": "You are ChatGPT, a narrative AI assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0
    )

    # completion에서 응답 추출
    # azure.ai.openai의 response 형식을 참고해야 함.
    # 일반적으로 completion.choices[0].message.content 형태로 접근 가능
    return completion.choices[0].message.content.strip()

def parse_time_range(time_range_str):
    match = re.search(r"(\d+)\s*~\s*(\d+)", time_range_str)
    if match:
        start = int(match.group(1))
        end = int(match.group(2))
        return start, end
    return None, None

def get_description_and_relationships(current_time, current_location, story_data, relationship_data):
    time_segments = story_data.get("time_segments", [])
    matched_segment = None
    for segment in time_segments:
        tr = segment.get("time_range", "")
        start, end = parse_time_range(tr)
        if start is not None and end is not None:
            if start <= current_time < end:
                matched_segment = segment
                break

    if not matched_segment:
        return "", [], []

    characters_info = matched_segment.get("characters", [])
    location_descriptions = []
    characters_in_location = set()

    for char_data in characters_info:
        name = char_data.get("name", "")
        actions = char_data.get("actions", [])
        for action in actions:
            locations = action.get("locations", [])
            if current_location in locations:
                desc = action.get("description", "")
                if desc:
                    location_descriptions.append(f"{name}: {desc}")
                characters_in_location.add(name)

    detailed_characters = []
    char_name_to_data = {}

    rel_characters = relationship_data.get("characters", [])
    for c in rel_characters:
        cname = c.get("name", "")
        if cname in characters_in_location:
            char_name_to_data[cname] = {
                "name": cname,
                "role": c.get("role", ""),
                "personality": c.get("personality", ""),
                "relationships": c.get("relationships", [])
            }

    characters_in_location_list = list(characters_in_location)
    relationship_descriptions = []
    for i in range(len(characters_in_location_list)):
        for j in range(i+1, len(characters_in_location_list)):
            char_a = characters_in_location_list[i]
            char_b = characters_in_location_list[j]
            data_a = char_name_to_data.get(char_a)
            data_b = char_name_to_data.get(char_b)
            if data_a and data_b:
                # a->b
                for rel in data_a["relationships"]:
                    related_chars = rel.get("characters", [])
                    if char_b in related_chars:
                        relationship_descriptions.append(f"{char_a} - {char_b}: {rel.get('description','')}")
                # b->a
                for rel in data_b["relationships"]:
                    related_chars = rel.get("characters", [])
                    if char_a in related_chars:
                        relationship_descriptions.append(f"{char_b} - {char_a}: {rel.get('description','')}")

    for c in characters_in_location_list:
        cdata = char_name_to_data.get(c, {})
        detailed_characters.append({
            "name": cdata.get("name", c),
            "role": cdata.get("role", ""),
            "personality": cdata.get("personality", "")
        })

    final_description = "\n".join(location_descriptions)
    return final_description, detailed_characters, relationship_descriptions

def show_intro():
    intro_path = os.path.join("Prompt", "intro_prompt.txt")
    with open(intro_path, "r", encoding="utf-8") as f:
        intro_text = f.read()
    print(intro_text)

def parse_openai_response(response_text: str):
    # current_time, current_location 추출은 더이상 필요 없음
    story_index = response_text.find("Story:")
    summary_index = response_text.find("summary:")

    if story_index != -1 and summary_index != -1:
        story_content = response_text[story_index+6:summary_index].strip()
    else:
        story_content = response_text

    if summary_index != -1:
        summary_content = response_text[summary_index+8:].strip()
    else:
        summary_content = ""

    # story_content와 summary_content만 반환
    return story_content, summary_content

def run_game_loop():
    with open("./json/story.json", "r", encoding="utf-8") as f:
        story_data = json.load(f)

    with open("./json/relationship.json", "r", encoding="utf-8") as f:
        relationship_data = json.load(f)

    show_intro()

    current_time = 0
    current_location = "1st floor entrance"
    summary = ""
    actions_taken = 0
    max_time = 50
    attempt = 1
    solved = False

    def build_prompt(ct, cl, summ, ua, desc, rel_str):
        return f"""
You are ChatGPT, a narrative AI that continues a detective story. Describe events and characters as if they truly exist.

[REFERENCE - DO NOT REVEAL]
Current Time: {ct} minutes
Current Location: {cl}
Summary:
{summ}

User Action:
{ua}

Description (from story_English.json, relevant events at current time/location):
{desc}

Relationships (for AI context only, do not reveal to user):
{rel_str}
[END OF REFERENCE]

Instructions:
- Use the provided Summary, User Action, and Description to continue the narrative realistically.
- The Relationships data is for internal consistency only; do not show or mention it.
- Do not reveal internal summaries, reasoning, or meta information. Only show the story continuation as a natural scene.
- After continuing the story, format the final output for the user as follows:

Output:
Story: [Reflect the user's action and the immediate narrative consequences]

Summary:[Updated summary of events for the user, reflecting the story progression]

If the user has solved the case or prevented the murder, write "Game ended" somewhere in the story to indicate the end.
Translate the output in Korean.
"""

    while True:
        user_action = input("\nYour action: ")
        actions_taken += 1

        # 사용자 액션 파싱
        # 예: "ask Lee Jae-hoon about the missing key,1st floor study"
        # 마지막에 콤마와 장소가 있으면 그 장소로 이동
        if "," in user_action:
            parts = user_action.rsplit(",", 1)  # 뒤에서부터 1회 분리
            action_part = parts[0].strip()
            location_part = parts[1].strip()
            # location_part를 최종 이동 장소로 설정
            current_location = location_part.capitalize()
            user_action = action_part  # 액션 부분만 남김
        else:
            # 콤마가 없다면 이동 없음, user_action 그대로 사용
            pass

        # 2번 행동마다 5분 경과
        if actions_taken % 2 == 0:
            current_time += 5

        desc, chars, rels = get_description_and_relationships(current_time, current_location, story_data, relationship_data)

        if desc == "" and chars == [] and rels == []:
            # 아무 일도 일어나지 않았을 경우
            print("\n아무 일도 일어나지 않았습니다.")
            # 프롬프트 전송 없이 다음 루프로
            # 시간은 이미 행동에 따라 증가함
            # 시도나 찬스 감소 로직 없음 (요구사항에 명시 없음)
            # 그냥 넘어감
            # 다음 action 입력 대기
            # 시간이 지나면 max_time 도달 시 처리
        else:
            summary += f"At {current_time} min, user did '{user_action}'.\n"
            rel_str = "\n".join(rels)
            prompt = build_prompt(current_time, current_location, summary, user_action, desc, rel_str)

            response_text = get_story_from_openai(prompt)
            story_content, new_summary = parse_openai_response(response_text)

            # 응답에서 "Game ended"가 포함되어 있으면 종료
            if "Game ended" in story_content:
                solved = True

            if new_summary:
                summary = new_summary

            print("\nOutput\n")
            print(f"Current Time: {current_time}")
            print(f"Current Location: {current_location}")
            print("Story:")
            print(story_content)


            if solved:
                print("\n축하합니다! 게임을 완료하셨습니다.")
                break

        # 시간이 최대에 도달했는데 해결 못했을 경우
        if current_time >= max_time and not solved:
            attempt += 1
            if attempt <= 3:
                max_time -= 10
                current_time = 0
                current_location = "1st floor entrance"
                summary = ""
                actions_taken = 0
                print(f"\n시간 초과! 새로운 시도로 돌아갑니다. 현재 가능한 최대 시간: {max_time}분")
            else:
                print("3번의 시도를 모두 소진하였습니다. Game Over.")
                break


if __name__ == "__main__":
    run_game_loop()


[Game Start Notice]

Welcome to the Detective Story Game. In this interactive narrative, you assume the role of a detective trying to either prevent a murder or uncover the murderer’s identity within a limited time.

Time will start at 0 minutes and can go up to 50 minutes. You have 3 total attempts (timelines) to solve the case. Every time you exhaust 3 actions, the available time decreases by 10 minutes (e.g., 0–50 minutes → 0–40 minutes → 0–30 minutes).

You can move to different locations in the mansion, talk to various characters, and inspect objects to gather clues. Each action you take will advance the story and time. You must be strategic in choosing locations and interacting with characters to find critical evidence and solve the mystery before time runs out.

Your goal:
- Identify the killer or prevent the murder before the allotted time expires.

Examples of locations you might visit:
['1st floor Kitchen', '1st floor Storage room', '1st floor entrance', '1st floor hallway', 

KeyboardInterrupt: Interrupted by user